In [1]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

pd.set_option("display.max.columns", 100)

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

In [5]:
train_df= pd.read_csv('data/train_reg.csv',index_col='Id')
test_df = pd.read_csv('data/test_reg.csv', index_col='Id')


submission = pd.read_csv('data/sample_submission_reg.csv')

In [8]:
from sklearn.model_selection import train_test_split
from catboost.utils import eval_metric

from catboost import Pool
from catboost import CatBoostRegressor
import numpy as np

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 230 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              800 non-null    float64
 1   Blast Furnace Slag  800 non-null    float64
 2   Fly Ash             800 non-null    float64
 3   Water               800 non-null    float64
 4   Superplasticizer    800 non-null    float64
 5   Coarse Aggregate    800 non-null    float64
 6   Fine Aggregate      800 non-null    float64
 7   Age                 800 non-null    int64  
 8   Strength            800 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 62.5 KB


In [21]:
target_columns = 'Strength'
numerical_columns = test_df.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = test_df.select_dtypes(include=['object']).columns

X_train, X_val, y_train, y_val = train_test_split(train_df.drop(target_columns, axis=1), train_df[target_columns], test_size=0.3, random_state=42)

cat_train_pool = Pool(X_train, y_train, cat_features=np.array(categorical_columns))
cat_val_pool = Pool(X_val, y_val, cat_features=np.array(categorical_columns)) 
cat_test_pool = Pool(test_df, cat_features=np.array(categorical_columns)) 
 

In [22]:
model_cat = CatBoostRegressor(
    verbose=100,
    random_seed=42)
model_cat.fit(cat_train_pool, eval_set=cat_val_pool)

Learning rate set to 0.04649
0:	learn: 16.9204424	test: 14.9357049	best: 14.9357049 (0)	total: 1.13ms	remaining: 1.13s
100:	learn: 4.7517623	test: 5.4173634	best: 5.4173634 (100)	total: 93.6ms	remaining: 833ms
200:	learn: 3.6205832	test: 4.8544292	best: 4.8544292 (200)	total: 176ms	remaining: 700ms
300:	learn: 2.8950120	test: 4.5477559	best: 4.5476032 (299)	total: 264ms	remaining: 614ms
400:	learn: 2.4495420	test: 4.3729538	best: 4.3729538 (400)	total: 372ms	remaining: 556ms
500:	learn: 2.1391001	test: 4.2670749	best: 4.2670749 (500)	total: 475ms	remaining: 473ms
600:	learn: 1.9168466	test: 4.1803293	best: 4.1803100 (599)	total: 566ms	remaining: 376ms
700:	learn: 1.7173162	test: 4.1284972	best: 4.1284972 (700)	total: 661ms	remaining: 282ms
800:	learn: 1.5504079	test: 4.0967600	best: 4.0967600 (800)	total: 760ms	remaining: 189ms
900:	learn: 1.4208804	test: 4.0709921	best: 4.0709921 (900)	total: 917ms	remaining: 101ms
999:	learn: 1.3189500	test: 4.0643980	best: 4.0639884 (966)	total: 1s	

In [25]:
submission[target_columns] = pd.Series(model_cat.predict(cat_test_pool))
submission.to_csv("submission_regression.csv", index=False)